PS D:\MS Purdue\1\ECE570> python --version

Python 3.9.0

PS D:\MS Purdue\1\ECE570> py -3.11 --version

Python 3.11.9

py -3.11 -m pip list

py -3.11 -m pip install <package_name>

In [22]:
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import os

# Print the current working directory
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

Current directory: e:\ECE570(E)\Project\1stAttempt


In [23]:
transform = transforms.Compose([
    # Resize to Tiny ImageNet's image size
    transforms.Resize((256, 256)),  
    transforms.ToTensor(),
    # Normalization
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalization
])

# Path to dataset
data_dir = '../cleaned_dataset/'

# Load training and validation data
train_dataset = datasets.ImageFolder(root=data_dir + 'train', transform=transform)
val_dataset = datasets.ImageFolder(root=data_dir + 'val', transform=transform)


# DataLoader for batching
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Check class-to-index mapping
print(train_dataset.class_to_idx) # Class labels
# Check class-to-index mapping and dataset size
print(train_dataset.class_to_idx)  # Class labels

print(f"Train subset size: {len(train_dataset)}")
print(f"Validation subset size: {len(val_dataset)}")

# Example: Iterate through the train_loader to test
for images, labels in train_loader:
    print(images.shape, labels.shape)
    break

{'n01440764': 0, 'n01496331': 1, 'n01530575': 2, 'n01558993': 3, 'n01601694': 4, 'n01629819': 5, 'n01641577': 6, 'n01667114': 7, 'n01682714': 8, 'n01692333': 9, 'n01698640': 10, 'n01729977': 11, 'n01740131': 12, 'n01751748': 13, 'n01770081': 14, 'n01774384': 15, 'n01795545': 16, 'n01806567': 17, 'n01820546': 18, 'n01843065': 19, 'n01860187': 20, 'n01882714': 21, 'n01924916': 22, 'n01950731': 23, 'n01980166': 24, 'n01986214': 25, 'n02007558': 26, 'n02013706': 27, 'n02027492': 28, 'n02056570': 29, 'n02077923': 30, 'n02086240': 31, 'n02088094': 32, 'n02089078': 33, 'n02090721': 34, 'n02091635': 35, 'n02093428': 36, 'n02094114': 37, 'n02095889': 38, 'n02096585': 39, 'n02097474': 40, 'n02099267': 41, 'n02100236': 42, 'n02101388': 43, 'n02102480': 44, 'n02105162': 45, 'n02105855': 46, 'n02106662': 47, 'n02107908': 48, 'n02108915': 49, 'n02110185': 50, 'n02111129': 51, 'n02112137': 52, 'n02113624': 53, 'n02114548': 54, 'n02116738': 55, 'n02120505': 56, 'n02124075': 57, 'n02128925': 58, 'n0213

In [24]:
# Initialize the model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Use GPU if available

# Import the specific functions from the vanillanet.py file
from vanillanet_NN import vanillanet_5, vanillanet_6, vanillanet_9

# Create an instance of the VanillaNet model (VanillaNet-9 in this case)
model = vanillanet_9(pretrained=False, num_classes=200).to(device)  

# Print model architecture to verify
print(model)


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 10.00 GiB already allocated; 0 bytes free; 10.00 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [4]:
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt  
from timm.optim import AdamP  

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = AdamP(model.parameters(), lr=0.001, weight_decay=0.35)  # LAMB or AdamP with weight decay

# Early Stopping Parameters
best_val_loss = float('inf')
patience = 5
counter = 0

# Initialize lists to store metrics
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

num_epochs = 10000  

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to device

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Update running loss
        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    # Calculate average training loss and accuracy
    avg_train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)

    # Validation loop
    model.eval()  # Set model to evaluation mode
    correct_val = 0
    total_val = 0
    val_loss = 0.0

    with torch.no_grad():  # No need to calculate gradients during validation
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    # Calculate average validation loss and accuracy
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct_val / total_val

    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)

    # Print epoch statistics
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
    print(f'Validation Accuracy: {val_accuracy:.2f}%, Validation Loss: {avg_val_loss:.4f}')


    if(epoch % 10 == 0):
        # Save checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': avg_train_loss,
            'val_loss': avg_val_loss,
            'train_accuracy': train_accuracy,
            'val_accuracy': val_accuracy,
        }
        torch.save(checkpoint, f'checkpoint_epoch_{epoch + 1}.pth')

    # Check for early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        counter = 0
        # Save the best model
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_val_loss,
        }, 'best_checkpoint.pth')
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered")
            break

print('Training complete!')

# Plotting only up to the epoch where training stopped
epochs = range(1, epoch + 2)  # Adjust the range to match the actual number of trained epochs

plt.figure(figsize=(12, 5))

# Plot training & validation loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, 'bo-', label='Train Loss')
plt.plot(epochs, val_losses, 'ro-', label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot training & validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, 'bo-', label='Train Accuracy')
plt.plot(epochs, val_accuracies, 'ro-', label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()

# Save the plot after training completion
plt.savefig('training_progress.png')
plt.show()


RuntimeError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 0; 4.00 GiB total capacity; 9.74 GiB already allocated; 0 bytes free; 9.97 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF